# Brain Tumor MRI Classification



# Table of Contents

* [Introduction](#introduction)
* [Dataset Description](#dataset-description)
* [Import Libraries](#import-libraries)
* [Define Dataset Paths](#define-dataset-paths)
* [Prepare Image Generators](#prepare-image-generators)
* [Build the Model (Transfer Learning)](#build-the-model)
* [Set Callbacks](#set-callbacks)
* [Train the Model](#train-the-model)
* [Evaluate the Model](#evaluate-the-model)
* [Save Model](#save-model)
* [Notes and Tips](#notes-and-tips)


### <a name="introduction"></a>Introduction
This notebook focuses on classifying brain tumors from MRI images using deep learning techniques.
We start with a baseline CNN model and then use DenseNet121 for transfer learning.
The dataset is sourced from Kaggle: [Brain Tumor MRI Dataset](https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset?resource=download).

Objectives:
- Train a CNN to classify MRI images into four tumor types.
- Apply data augmentation to reduce overfitting.
- Use transfer learning with DenseNet121 for higher accuracy.
- Fine-tune top layers to improve model performance.

### <a name="dataset-description"></a>Dataset Description

The dataset used in this project consists of MRI brain images categorized into four classes:

| Class Name     | Number of Images |
| -------------- | ---------------- |
| **No Tumor**   | 1,595            |
| **Glioma**     | 1,321            |
| **Meningioma** | 1,339            |
| **Pituitary**  | 1,457            |
| **Total**      | **5,712**        |


Originally, the dataset was divided into two main folders:

* train

* test

However, in this notebook, the training dataset was further divided into training and validation subsets using validation_split=0.2.

The final data distribution after splitting is as follows:

* Training set: 4,571 images (4 classes)

* Validation set: 1,141 images (4 classes)

* Test set: 1,311 images (4 classes)

This dataset is sourced from the Kaggle Brain Tumor MRI Dataset, which contains MRI scans labeled according to the tumor type.

https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset?resource=download

### <a name="import-libraries"></a>Import Libraries


In [ ]:
import tensorflow as tf
tf.keras.mixed_precision.set_global_policy("float32")

from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import numpy as np
import os

In [ ]:
import os, zipfile
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip_path = '/content/drive/MyDrive/mri-data.zip'
extract_path = '/content/mri-data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

base_dir = '/content/mri-data'
train_dir = os.path.join(base_dir, 'Training')
test_dir  = os.path.join(base_dir, 'Testing')

print("Data extracted successfully")
print("Train path:", train_dir)
print("Test path:", test_dir)


Data extracted successfully
Train path: /content/mri-data/Training
Test path: /content/mri-data/Testing


### <a name="define-dataset-paths"></a>Define Dataset Paths



In [ ]:
train_dir = "/content/mri-data/Training"
test_dir = "/content/mri-data/Testing"

### <a name="prepare-image-generators"></a>Prepare Image Generators


In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    zoom_range=0.2,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    validation_split=0.2
)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

validation_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

print("Class Order:", train_data.class_indices)


Found 4571 images belonging to 4 classes.
Found 1141 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.
Class Order: {'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}


### <a name="build-the-model"></a>Build the Model (Transfer Learning)


### DenseNet121

In [ ]:
input_layer = Input(shape=(224, 224, 3))

base = DenseNet121(include_top=False, weights="imagenet", input_tensor=input_layer)
base.trainable = False

x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.4)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
output_layer = Dense(4, activation="softmax")(x)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer=Adam(1e-3), loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_2    │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,408 │ zero_padding2d_2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_3    │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 56, 56,    │          0 │ zero_padding2d_3… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 56, 56,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 56, 56,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 56, 56,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 56, 56,    │     12,288 │ conv2_block2_0_r

 Total params: 7,300,932 (27.85 MB)

 Trainable params: 263,428 (1.00 MB)

 Non-trainable params: 7,037,504 (26.85 MB)

### <a name="set-callbacks"></a>Set Callbacks


In [ ]:
checkpoint = ModelCheckpoint("brain_model.keras", save_best_only=True, monitor="val_accuracy")
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

### <a name="train-the-model"></a>Train the Model


In [ ]:
model.fit(train_data,
          validation_data=validation_data,
          epochs=6,
          callbacks=[checkpoint, lr_reduce, early_stop])

Epoch 1/6
286/286 ━━━━━━━━━━━━━━━━━━━━ 143s 408ms/step - accuracy: 0.6248 - loss: 1.0194 - val_accuracy: 0.6398 - val_loss: 0.8292 - learning_rate: 0.0010
Epoch 2/6
286/286 ━━━━━━━━━━━━━━━━━━━━ 78s 274ms/step - accuracy: 0.7955 - loss: 0.5261 - val_accuracy: 0.7651 - val_loss: 0.6129 - learning_rate: 0.0010
Epoch 3/6
286/286 ━━━━━━━━━━━━━━━━━━━━ 77s 269ms/step - accuracy: 0.8168 - loss: 0.4757 - val_accuracy: 0.7572 - val_loss: 0.6375 - learning_rate: 0.0010
Epoch 4/6
286/286 ━━━━━━━━━━━━━━━━━━━━ 75s 263ms/step - accuracy: 0.8292 - loss: 0.4391 - val_accuracy: 0.7423 - val_loss: 0.7259 - learning_rate: 0.0010
Epoch 5/6
286/286 ━━━━━━━━━━━━━━━━━━━━ 77s 268ms/step - accuracy: 0.8511 - loss: 0.4091 - val_accuracy: 0.7870 - val_loss: 0.5482 - learning_rate: 5.0000e-04
Epoch 6/6
286/286 ━━━━━━━━━━━━━━━━━━━━ 75s 264ms/step - accuracy: 0.8422 - loss: 0.4090 - val_accuracy: 0.7791 - val_loss: 0.5819 - learning_rate: 5.0000e-04


In [ ]:
for layer in base.layers[-60:]:
    layer.trainable = True

model.compile(optimizer=Adam(1e-4),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.fit(train_data,
          validation_data=validation_data,
          epochs=8,
          callbacks=[checkpoint, lr_reduce, early_stop])

Epoch 1/8
286/286 ━━━━━━━━━━━━━━━━━━━━ 161s 426ms/step - accuracy: 0.8345 - loss: 0.4581 - val_accuracy: 0.8186 - val_loss: 0.4685 - learning_rate: 1.0000e-04
Epoch 2/8
286/286 ━━━━━━━━━━━━━━━━━━━━ 79s 275ms/step - accuracy: 0.8896 - loss: 0.3031 - val_accuracy: 0.8440 - val_loss: 0.4192 - learning_rate: 1.0000e-04
Epoch 3/8
286/286 ━━━━━━━━━━━━━━━━━━━━ 78s 272ms/step - accuracy: 0.9068 - loss: 0.2384 - val_accuracy: 0.8869 - val_loss: 0.3450 - learning_rate: 1.0000e-04
Epoch 4/8
286/286 ━━━━━━━━━━━━━━━━━━━━ 81s 268ms/step - accuracy: 0.9171 - loss: 0.2354 - val_accuracy: 0.8764 - val_loss: 0.3397 - learning_rate: 1.0000e-04
Epoch 5/8
286/286 ━━━━━━━━━━━━━━━━━━━━ 78s 272ms/step - accuracy: 0.9313 - loss: 0.1804 - val_accuracy: 0.9132 - val_loss: 0.2673 - learning_rate: 1.0000e-04
Epoch 6/8
286/286 ━━━━━━━━━━━━━━━━━━━━ 78s 273ms/step - accuracy: 0.9386 - loss: 0.1652 - val_accuracy: 0.9238 - val_loss: 0.2732 - learning_rate: 1.0000e-04
Epoch 7/8
286/286 ━━━━━━━━━━━━━━━━━━━━ 77s 268ms/st

### <a name="evaluate-the-model"></a>Evaluate on Test Data



In [ ]:
print("Evaluating...")
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy = {test_acc:.4f}")

Evaluating...
82/82 ━━━━━━━━━━━━━━━━━━━━ 17s 212ms/step - accuracy: 0.9322 - loss: 0.1844
Test Accuracy = 0.9336


### <a name="Save Model"></a>Save Model


In [ ]:
model.save("brain_mri_model.keras", save_format="keras")

In [ ]:
# Save
model.save('/content/MRI_brain_Model_FUNCTIONAL.h5')
print("Model saved as MRI_brain_Model_FUNCTIONAL.h5")

Model saved as MRI_brain_Model_FUNCTIONAL.h5


In [ ]:
model.save("brain_model.keras")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/MRI_brain_Model_FUNCTIONAL.h5 "/content/drive/MyDrive/MRI_brain_Model_FUNCTIONAL.h5"

print("Model copied to Google Drive successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model copied to Google Drive successfully!


### <a name="notes-and-tips"></a>Notes and Tips
- Mixed precision speeds up training on compatible GPUs (e.g., A100).  
- Use strong data augmentation to reduce overfitting.  
- Save models frequently and copy to Google Drive for backup.  
- Fine-tuning improves accuracy but may lead to overfitting if not monitored.  
- Always monitor training vs validation loss to adjust augmentation, learning rates, or batch sizes.  
